In [0]:
from __future__ import print_function

In [0]:
import umap

umap3d = umap.UMAP(n_components=3, n_neighbors = 15, min_dist=0.1, metric = 'minkowski', random_state = 31)
umap2d = umap.UMAP(n_components=2, n_neighbors = 15, min_dist=0.1, metric = 'minkowski', random_state = 31)

In [3]:
import getpass
user = getpass.getuser()

if user == 'root':
    from google.colab import drive
    drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
################################################################
# import blocks
################################################################
# I/O
import os

# agrs
import sys

# load json
import json

# garabage collection
import gc

import numpy as np

import keras

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

# split folds
from sklearn.model_selection import KFold

# tsne setups
from sklearn.manifold import TSNE

t_steps = 600
t_perplexity = 30
t_learning_rate = 150
tsne2 = TSNE(n_components=2,
             n_iter=t_steps,
             n_iter_without_progress=t_steps,
             perplexity=t_perplexity,
             learning_rate = t_learning_rate,
             random_state = 31)
tsne3 = TSNE(n_components=3,
             n_iter=t_steps,
             perplexity=t_perplexity,
             n_iter_without_progress=t_steps,
             learning_rate = t_learning_rate,
             random_state = 31)

# plotting parameters
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.pyplot as plt
import statistics, math, numbers, numpy
import matplotlib.colors as colors
from operator import itemgetter


Using TensorFlow backend.


In [0]:
import umap

umap3 = umap.UMAP(n_components=3, n_neighbors = 15, min_dist=0.1, metric = 'minkowski', random_state = 31)
umap2 = umap.UMAP(n_components=2, n_neighbors = 15, min_dist=0.1, metric = 'minkowski', random_state = 31)

In [0]:
################################################################
# parameters setting
################################################################
# fix random seeds
from numpy.random import seed
seed(3)

from tensorflow import set_random_seed
set_random_seed(31)


BATCH_SIZE=32
EPOCHS=200
VALIDATION_SPLIT=0.2
TARGET_LAYER = -2
VIS_NUM_POINTS = 1000

challenges = {
    # QA1 with 10,000 samples, single_supporting_fact_10k
    'babi-q1': 'tasks_1-20_v1-2/en-10k/qa1_' 
                                  'single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples, two_supporting_facts_10k
    'babi-q2': 'tasks_1-20_v1-2/en-10k/qa2_'
                                'two-supporting-facts_{}.txt',
}

DATA_SET = challenge_type = sys.argv[1] if len(sys.argv) > 1 and sys.argv[1] != '-f' else 'babi-q1'
challenge = challenges[challenge_type]

# check points
path_header = '/content/gdrive/My Drive/Colab Notebooks/babi-memnn/' if user == 'root' else ''
checkpoint_path = path_header + 'saved_models/' + DATA_SET + '-e{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# data path
data_dir = path_header + 'saved_projections'

# colors
tableau10 = ["#4E79A7", "#F28E2B", "#E15759", "#76B7B2", "#59A14F",
            "#EDC948", "#B07AA1", "#FF9DA7", "#9C755F", "#BAB0AC"]

tableau20 = ["#4E79A7", "#A0CBE8", "#F28E2B", "#FFBE7D", "#59A14F",
             "#8CF17D", "#B6992D", "#EDC948",  "#499894", "#76B7B2",
             "#E15759", "#FF9D9A", "#79706E", "#BAB0AC", "#D37295",
             "#FABFD2", "#B07AA1", "#D4A6C8", "#9D7660", "#D7B5A6"]

tableau22 = ["#4E79A7", "#A0CBE8", "#F28E2B", "#FFBE7D", "#59A14F",
             "#8CF17D", "#B6992D", "#EDC948",  "#499894", "#76B7B2",
             "#E15759", "#FF9D9A", "#79706E", "#BAB0AC", "#D37295",
             "#FABFD2", "#B07AA1", "#D4A6C8", "#9D7660", "#D7B5A6", 
             "#FF9DA7", "#9C755F"]

tableau10_colors = colors.ListedColormap(tableau10)
tableau20_colors = colors.ListedColormap(tableau20)
tableau22_colors = colors.ListedColormap(tableau22)

In [0]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data
            if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        answers.append(word_idx[answer])
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))



In [8]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz',
                    origin='https://s3.amazonaws.com/text-datasets/'
                           'babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2'
          '.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise

11747328/11745123 [==============================] - 1s 0us/step


In [9]:
################################################################
# processing data
################################################################

print('Extracting stories for the challenge:', challenge_type)
with tarfile.open(path) as tar:
    train_stories = get_stories(tar.extractfile(challenge.format('train')))
    test_stories = get_stories(tar.extractfile(challenge.format('test')))

vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

Extracting stories for the challenge: babi-q1


/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000,)
answers_test shape: (1000,)
-
Compiling...


In [0]:
# load training history
with open(checkpoint_dir + '/' + DATA_SET + '-history.json') as f:
    hist_json = json.load(f)
    history = hist_json['history']


In [11]:
NUM_CLASSES = len(set(answers_test))
print(NUM_CLASSES)
desc_num_points = str(int(VIS_NUM_POINTS / 1000)) + 'k'

6


In [0]:
p2 = umap2
p3 = umap3
p_name = 'umap'

In [0]:
def write_file(content, path):
    file = open(path,'w+')
    file.write(content)
    file.close()
   
    
def write_fig(p2d, p3d, labels, fig_path):
    save_fig = plt.gcf()
    fig = plt.figure(figsize=(15, 7))

    ax = fig.add_subplot(1, 2, 1, projection='3d')

    ax.scatter(xs = p3d[:,0],
           ys = p3d[:,1],
           zs = p3d[:,2],
           c = labels,
           cmap = tableau10_colors, alpha = 1)
    plt.title(p_name + ' 3D after training')

    fig.add_subplot(1, 2, 2)

    plt.scatter(x = p2d[:,0],
            y = p2d[:,1],
            c = labels,
            cmap = tableau10_colors, alpha= 1)
    plt.title(p_name + ' 2D after training')

    plt.draw()
    fig.savefig(fig_path, dpi=100)
    plt.close(fig)

In [14]:
################################################################
#compute projections
################################################################

file_names = ""
test_acc_records = ""
training_acc_records = ""

    
x_ = inputs_test
q_ = queries_test
y_ = answers_test
j = 0
y_labels = y_.copy()

# for each model
for e in np.arange(184, EPOCHS):
    
    model_path = checkpoint_dir + '/' + DATA_SET + '-e%04d.ckpt' % (e + 1)
    file_name = DATA_SET + '-e%04d'% (e + 1) + '-f%d'%j + '-' + desc_num_points + '-l%d'%TARGET_LAYER + '-' + p_name + '.txt'
    fig_path = path_header + 'saved_figs/' + DATA_SET + '-e%04d'% (e + 1) + '-f%d'%j + '-' + desc_num_points + '-l%d'%TARGET_LAYER + '-' + p_name + '.png'

    # load model
    print('.........loading ' + model_path + ' for ' + p_name)
    saved_model = keras.models.load_model(model_path)
    
    target_layer_model = Model(inputs=saved_model.input,
                               outputs=saved_model.layers[TARGET_LAYER].output)

    
    y_target = target_layer_model.predict([x_, q_])
    score = saved_model.evaluate([x_, q_], y_, verbose = 0, batch_size=BATCH_SIZE)
 
    y_pred = saved_model.predict([x_, q_], batch_size=BATCH_SIZE)
    y_pred_labels = [int(np.argmax(y)) for y in y_pred]
    y_correctness = list(map(lambda x, y: True if x == y else False, y_pred_labels, y_labels))
    
    
    y_target_p2d = p2.fit_transform(y_target)
    y_target_p3d = p3.fit_transform(y_target)
    
    #print('t-SNE steps %d, %d' % (tsne2.n_iter_ , tsne3.n_iter_))
    #print('t-SNE errors %f, %f' % (tsne2.kl_divergence_ , tsne3.kl_divergence_))

    # normalizing
    y_target_p2d_normed = y_target_p2d.transpose()
    y_target_p2d_normed = numpy.array([[(d - numpy.mean(d))/ numpy.std(d)] for d in y_target_p2d_normed]).reshape(2, VIS_NUM_POINTS).transpose()

    y_target_p3d_normed = y_target_p3d.transpose()
    y_target_p3d_normed = numpy.array([[(d - numpy.mean(d))/ numpy.std(d)] for d in y_target_p3d_normed]).reshape(3, VIS_NUM_POINTS).transpose()
    

    output_path = data_dir + '/' + file_name
    OUTPUT = ""

    OUTPUT += 'data_set\n' + DATA_SET + '\n' + \
    'num_classes\n' + str(NUM_CLASSES) + '\n' + \
    'data_points\n' + str(VIS_NUM_POINTS) + '\n' +\
     \
    'model\n' + model_path + '\n' +\
     \
    'color_schema\n' + ','.join(tableau10) + '\n' +\
    \
    'test_accuracy\n' + str(score[1]) + '\n' +\
    'test_loss\n' + str(score[0]) + '\n' +\
    \
    'train_accuracy\n' + str(history['acc'][e]) + '\n' +\
    'train_loss\n' + str(history['loss'][e]) + '\n' +\
    \
    'val_accuracy\n' + str(history['val_acc'][e]) + '\n' +\
    'val_loss\n' + str(history['val_loss'][e]) + '\n' +\
    \
    'labels\n' + ','.join([str(d) for d in y_labels]) + '\n' +\
    'pred_labels\n' + ','.join([str(d) for d in y_pred_labels]) + '\n' +\
    'pred_correctness\n' + ','.join([str(d) for d in y_correctness]) + '\n' +\
    \
    'p2d\n' + ';'.join([str(d[0]) + ',' + str(d[1]) for d in y_target_p2d_normed])  + '\n' +\
    \
    'p3d\n' + ';'.join([str(d[0]) + ',' + str(d[1])  + ',' + str(d[2]) for d in y_target_p3d_normed])  + '\n'

    file_names += file_name + ','
    test_acc_records += str(score[1]) + ','
    training_acc_records +=  str(history['acc'][e]) + ','
        
    file = open(output_path,'w+')
    file.write(OUTPUT)
    file.close()

    write_file(OUTPUT, output_path)
    write_fig(y_target_p2d_normed, y_target_p3d_normed, y_labels, fig_path)
        
    del y_target, y_pred, y_pred_labels, y_correctness, y_target_p2d, y_target_p3d, y_target_p2d_normed, y_target_p3d_normed
    gc.collect()
    
    
# write accuracy files
accuracy_path = data_dir + '/'+ 'accuracy.txt'
accuracy_content = file_names[:-1] + '\n' + test_acc_records[:-1] +'\n' + training_acc_records[:-1]
write_file(accuracy_content, accuracy_path)

.........loading /content/gdrive/My Drive/Colab Notebooks/babi-memnn/saved_models/babi-q1-e0185.ckpt for umap
.........loading /content/gdrive/My Drive/Colab Notebooks/babi-memnn/saved_models/babi-q1-e0186.ckpt for umap
.........loading /content/gdrive/My Drive/Colab Notebooks/babi-memnn/saved_models/babi-q1-e0187.ckpt for umap
.........loading /content/gdrive/My Drive/Colab Notebooks/babi-memnn/saved_models/babi-q1-e0188.ckpt for umap
.........loading /content/gdrive/My Drive/Colab Notebooks/babi-memnn/saved_models/babi-q1-e0189.ckpt for umap
.........loading /content/gdrive/My Drive/Colab Notebooks/babi-memnn/saved_models/babi-q1-e0190.ckpt for umap
.........loading /content/gdrive/My Drive/Colab Notebooks/babi-memnn/saved_models/babi-q1-e0191.ckpt for umap
.........loading /content/gdrive/My Drive/Colab Notebooks/babi-memnn/saved_models/babi-q1-e0192.ckpt for umap
.........loading /content/gdrive/My Drive/Colab Notebooks/babi-memnn/saved_models/babi-q1-e0193.ckpt for umap
.........l